In [1]:
import itertools
import random
import copy
import time
import pandas as pd
import numpy as np
import gc
import re
import torch

#import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')
from collections import Counter

from nltk import word_tokenize

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from sklearn.metrics import f1_score
import os 

#from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences

# cross validation and metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer

from sklearn.preprocessing import StandardScaler
from multiprocessing import  Pool
from functools import partial
import numpy as np
from sklearn.decomposition import PCA
import torch as t
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


In [2]:
#%pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
#%pip install -U scikit-learn scipy matplotlib
#%pip install tqdm
#%pip install nltk

In [3]:
%pip install xlrd
%pip install openpyxl
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import pandas as pd
import re
from sklearn.svm import LinearSVC
#from sklearn import svm 
from sklearn.calibration import CalibratedClassifierCV
from nltk.corpus import stopwords
import time
from nltk.stem import WordNetLemmatizer
import random
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import metrics
from datetime import datetime
from ast import literal_eval
#from torchtext import data



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')

In [7]:
# load the pre-processed train and test data
# LoadData...ipynb should have been used as a first step to process the text

df_train = pd.read_csv('df_train_shuffle_processed.csv')
df_train = df_train.sample(frac=1)# shuffle
test_all = df_train.iloc[-200:, :]# set aside the test data
#df_train = df_train.iloc[:-200, :]# split off the train from the test data

In [8]:
#%pip install gensim
df_train.text

5846    Ashley and I on going to hurricane harbor Frid...
632     Dr. Bengston on #wildfire management: ÛÏnumbe...
7144    Govt allocating 1.3 bn for flood action: Issue...
1024    And please don't flood poor @RobertBEnglund's ...
3230    It's cold and my head wants to explode.. The j...
                              ...                        
5969    'Nobody remembers who came in second.' Charles...
7548    @Michael5SOS FUCKING LIVE HERE IM SURPRISED IV...
1232    Busty blonde teen Natalia Starr fucks the secu...
1219    Malaysia seem more certain than France.\n\nPla...
6283    I will never support looting or the burning of...
Name: text, Length: 7613, dtype: object

In [9]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.corpora

gensim_model = Word2Vec.load("word2vec.model")
# load embeddings generated from Gensim script in separate notebook
myembeddings = {i:j for i,j in zip(gensim_model.wv.index_to_key, range(len(gensim_model.wv.index_to_key)))}
print(len(myembeddings))
myembeddings

19904


{'co': 0,
 'http': 1,
 'get': 2,
 'https': 3,
 'like': 4,
 'fire': 5,
 'û_': 6,
 'amp': 7,
 'go': 8,
 'bomb': 9,
 'new': 10,
 'via': 11,
 '2': 12,
 'people': 13,
 'say': 14,
 'one': 15,
 'news': 16,
 'burn': 17,
 'kill': 18,
 'make': 19,
 'time': 20,
 'flood': 21,
 'video': 22,
 'crash': 23,
 'come': 24,
 'disaster': 25,
 'emergency': 26,
 'build': 27,
 'body': 28,
 'attack': 29,
 'see': 30,
 'look': 31,
 'home': 32,
 'police': 33,
 'take': 34,
 'know': 35,
 'think': 36,
 'would': 37,
 'u': 38,
 'love': 39,
 '3': 40,
 'still': 41,
 'train': 42,
 'storm': 43,
 'back': 44,
 'us': 45,
 'california': 46,
 'bag': 47,
 'suicide': 48,
 'want': 49,
 'collapse': 50,
 'watch': 51,
 'live': 52,
 'day': 53,
 'man': 54,
 'scream': 55,
 '1': 56,
 'work': 57,
 'first': 58,
 'rt': 59,
 'cause': 60,
 'let': 61,
 'world': 62,
 'nuclear': 63,
 'two': 64,
 'need': 65,
 'war': 66,
 'drown': 67,
 'today': 68,
 'û': 69,
 'wreck': 70,
 'youtube': 71,
 'year': 72,
 'dead': 73,
 'plan': 74,
 'destroy': 75,
 '5'

In [10]:
# check tet and train dimensions
print(test_all.shape)
print(df_train.shape)

(200, 7)
(7613, 7)


In [11]:
# set aside the text corpus
description = [literal_eval(s) for s in df_train.description.tolist()]
df_train.description = description
# PyTorch NN needs a vocab list, or unique vocab terms
vocab = []
vocab = gensim_model.wv.index_to_key#list(set(vocab))

In [12]:
# make the train tensors for text to embeddings X and for categories/products Y
x_train = [[myembeddings[j] for j in i] for i in description]

In [13]:
# make the test tensors for text to embeddings X and for categories/products Y
test_description = [literal_eval(s) for s in test_all.description.tolist()]

x_val = []
for i in test_description:
    temp = []
    for j in i:
        # some of the Amazon tokens don't appear in the training data
        # so use try/except to skip any unaccounted for Amazon test data tokens
        try: temp.append(myembeddings[j])
        except: pass
    x_val.append(temp)

In [14]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
#device

In [15]:
vocab = vocab 
num_class = 1
vocab_size = len(vocab)
print(vocab_size)

19904


In [16]:
maxlen = 120

In [17]:
new_xt = []
for x in x_train:
    if len(x) > maxlen: new_xt.append(np.array(x[:maxlen]))
    else: new_xt.append(np.array(list(x) + [0]*(maxlen-len(x)) ) ) #concatenate list with padding zeros and convert to array
new_xt = np.array(new_xt)        

In [18]:
# the arrays should be the same length
print(len(new_xt),  len(x_train))
len(new_xt) ==  len(x_train)

7613 7613


True

In [19]:
new_xv = []
for x in x_val:
    if len(x) > maxlen: new_xv.append(np.array(x[:maxlen]))
    else: new_xv.append(np.array(list(x) + [0]*(maxlen-len(x)) )  )
    
new_xv = np.array(new_xv)
        

In [20]:
x_train = torch.tensor(new_xt, dtype=torch.long)
x_cv = torch.tensor(new_xv, dtype=torch.long)

In [21]:
vocab_size = len(vocab)
print(vocab_size)
embed_size = 200 # how big is each word vector
max_features = vocab_size
batch_size = 64 # how many samples to process at once
n_splits = 8 # Number of K-fold Splits
SEED = 10
debug = 0

19904


## Pytorch Model - TextCNN
based on https://www.kaggle.com/code/mlwhiz/multiclass-text-classification-pytorch
but with different loss function loss_fn = torch.nn.BCEWithLogitsLoss()
so that the results can be multi-label instead of multi-class,
meaning that there can be multiple products per narrative.

In [22]:
class CNN_Text(nn.Module):
    
    def __init__(self, num_class):
        super(CNN_Text, self).__init__()
        filter_sizes = [1,2,3,5]
        num_filters = 128
        #n_classes = len(le.classes_)
        n_classes = num_class
        self.embedding = nn.Embedding(max_features, embed_size)
        #self.embedding.weight = nn.Parameter(torch.tensor(gensim_model.wv.syn0, dtype=torch.float32))
        #self.embedding.weight.requires_grad = False
        self.convs1 = nn.ModuleList([nn.Conv2d(1, num_filters, (K, embed_size)) for K in filter_sizes])
        self.dropout = nn.Dropout(0.3)#nn.Dropout(0.2)
        self.fc1 = nn.Linear(len(filter_sizes)*num_filters, n_classes)
        self.elu = nn.ELU()
        self.silu = nn.SiLU()
        self.softplus = nn.Softplus()

    def forward(self, x):
        x = self.embedding(x)  
        x = x.unsqueeze(1)  
        #x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1] 
        x = [F.elu(conv(x)).squeeze(3) for conv in self.convs1] 
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  
        x = torch.cat(x, 1)
        x = self.dropout(x)  
        logit = self.fc1(x)

        return logit

In [23]:
class BiLSTM(nn.Module):
    
    def __init__(self, num_class):
        super(BiLSTM, self).__init__()
        self.hidden_size = 128
        drp = 0.4
        n_classes = num_class#len(le.classes_)
        self.embedding = nn.Embedding(max_features, embed_size)
        #self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        #self.embedding.weight.requires_grad = False
        #self.embedding.weight = nn.Parameter(torch.tensor(gensim_model.wv.syn0, dtype=torch.float32))
        #self.embedding.weight.requires_grad = False
        self.lstm = nn.LSTM(embed_size, self.hidden_size, bidirectional=True, batch_first=True)
        self.linear = nn.Linear(4*self.hidden_size, 6*self.hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)
        self.linear2 = nn.Linear(6*self.hidden_size, 3*self.hidden_size)
        self.out = nn.Linear(3*self.hidden_size, n_classes)
        
        self.elu = nn.ELU()
        self.silu = nn.SiLU()
        self.softplus = nn.Softplus()

    def forward(self, x):
        #rint(x.size())
        h_embedding = self.embedding(x)
        h_embedding = self.elu(h_embedding)
        #_embedding = torch.squeeze(torch.unsqueeze(h_embedding, 0))
        h_lstm, _ = self.lstm(h_embedding)
        avg_pool = torch.mean(h_lstm, 1)
        max_pool, _ = torch.max(h_lstm, 1)
        conc = torch.cat(( avg_pool, max_pool), 1)
        
        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)
        conc = self.softplus(self.linear2(conc))# extra hidden layer
        conc = self.dropout(conc)
        out = self.out(conc)
#        out = self.softplus(out)
        return out

## Pytorch Model - BiLSTM

In [24]:
print(CNN_Text(num_class))
print(BiLSTM(num_class))

CNN_Text(
  (embedding): Embedding(19904, 200)
  (convs1): ModuleList(
    (0): Conv2d(1, 128, kernel_size=(1, 200), stride=(1, 1))
    (1): Conv2d(1, 128, kernel_size=(2, 200), stride=(1, 1))
    (2): Conv2d(1, 128, kernel_size=(3, 200), stride=(1, 1))
    (3): Conv2d(1, 128, kernel_size=(5, 200), stride=(1, 1))
  )
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=512, out_features=1, bias=True)
  (elu): ELU(alpha=1.0)
  (silu): SiLU()
  (softplus): Softplus(beta=1, threshold=20)
)
BiLSTM(
  (embedding): Embedding(19904, 200)
  (lstm): LSTM(200, 128, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=768, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.4, inplace=False)
  (linear2): Linear(in_features=768, out_features=384, bias=True)
  (out): Linear(in_features=384, out_features=1, bias=True)
  (elu): ELU(alpha=1.0)
  (silu): SiLU()
  (softplus): Softplus(beta=1, threshold=20)
)


## Train NN Model

In [25]:
def train_model(model_type='cnn'):

    num_class = 2# number of classes is the highest number class plus 1 because class keys start at 0
    print(num_class,'labels')
    
    new_yt = []# train
    for y in df_train.target.tolist():
        temp = np.zeros(num_class)# instantiate output nodes for that entry
        temp[y] = 1# look up the category in the dictionary and use that as the index set to positive (1) in the output array
        new_yt.append(temp)#append output nodes to the ylist

    new_yv = []# test / val
    for y in test_all.target.tolist():
        temp = np.zeros(num_class)# instantiate output nodes for that entry
        temp[y] = 1# look up the category in the dictionary and use that as the index set to positive (1) in the output array
        new_yv.append(temp)#append output nodes to the ylist
    
    y_train = torch.tensor(new_yt)#[torch.tensor(i).cuda() for i in new_yt]#, dtype=torch.long).cuda()
    y_cv = torch.tensor(new_yv)
    #print(new_yt[0], new_yv[0])
    # Create Torch datasets
    train = torch.utils.data.TensorDataset(x_train, y_train)
    valid = torch.utils.data.TensorDataset(x_cv, y_cv)
    
    batch_size = 128

    # use loaders to iterate through data batches during training
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
    
    n_epochs = 2# number of epochs
    # one function that can do either CNN or LSTM
    if 'cnn' in model_type.lower(): 
        model = CNN_Text(num_class)
        gamma_val = 0.05# low learning rate for CNN
    elif 'lstm' in model_type.lower(): 
        model = BiLSTM(num_class)
        gamma_val = 0.3# higher learning rate for LSTM
    
    model
    loss_fn = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.02)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=gamma_val)# gamma val goes here, what the LR is multiplied by per iteration by the scheduler

    train_loss = []
    valid_loss = []

    for epoch in range(n_epochs):
        start_time = time.time()
        # Set model to train configuration
        model.train()
        avg_loss = 0.  
        for i, (x_batch, y_batch) in enumerate(train_loader):
            # Predict/Forward Pass
            y_pred = model(x_batch)
            # Compute loss
            loss = loss_fn(y_pred, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            avg_loss += loss.item() / len(train_loader)
            #print(loss)

        # Set model to validation configuration -Doesn't get trained here
        model.eval()        
        avg_val_loss = 0.
        #val_preds = np.zeros((len(x_cv),len(le.classes_)))
        val_preds = np.zeros((len(x_cv),num_class))

        for i, (x_batch, y_batch) in enumerate(valid_loader):
            y_pred = model(x_batch).detach()
            avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
            # keep/store predictions
            val_preds[i * batch_size:(i+1) * batch_size] = F.softmax(y_pred).cpu().numpy()#y_pred.cpu().numpy()#F.softmax(y_pred).cpu().numpy()
            #val_preds[i * batch_size:(i+1) * batch_size] = F.sigmoid(y_pred).cpu().numpy()#y_pred.cpu().numpy()#F.softmax(y_pred).cpu().numpy()

    # Check Accuracy
        pred_ = [np.where(i>0.1) for i in val_preds]#[np.where(i>0) for i in val_preds]
        val_ = [np.where(i==1) for i in new_yv]
        # tps are elements j of predicted lists that are in the val lists, list by list, i,k,
        tps = 0
        fps = 0
        fns = 0
        for i,k in zip(pred_, val_):
            if len(i[0]) > 0:
                for j in i:
                    if j[0] in list(k[0]): tps+=1
                    elif j[0] not in list(k[0]): fps+=1
            if len(k[0]) > 0:
                for m in k:
                    if m[0] not in list(i[0]): fns+=1

        try: precision = tps/(tps+fps)
        except: precision = 0
        try: recall = tps/(tps+fns)
        except: recall = 0
        try: f1 = (2 * precision * recall) / (precision + recall)
        except: f1 = 0
        print(round(precision,4), 'precision.', round(recall,4), 'recall.', round(f1,4), 'F1.')
        val_accuracy = f1#sum(val_preds.argmax(axis=1)==new_yv)/len(new_yv)
        train_loss.append(avg_loss)
        valid_loss.append(avg_val_loss)
        elapsed_time = time.time() - start_time 
        print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f}  \t val_acc={:.4f}  \t time={:.2f}s'.format(
                    epoch + 1, n_epochs, avg_loss, avg_val_loss, val_accuracy, elapsed_time))

        scheduler.step()
    torch.save(model,'{}_model'.format(model_type))

In [26]:
# this is the generic category level and the cat_selection variable is just a dummy variable
start_time = time.time()
train_model(model_type='cnn')
duration = round((time.time() - start_time)/60/60, 2)# duration in seconds, round to reasonable decimal
print("this loop took {} hours to run".format(duration))# print time it took, seconds converted to minutes to hours


2 labels


C:\Users\Owner\AppData\Local\Temp\ipykernel_18476\1857778842.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:204.)
  y_train = torch.tensor(new_yt)#[torch.tensor(i).cuda() for i in new_yt]#, dtype=torch.long).cuda()
C:\Users\Owner\AppData\Local\Temp\ipykernel_18476\1857778842.py:74: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  val_preds[i * batch_size:(i+1) * batch_size] = F.softmax(y_pred).cpu().numpy()#y_pred.cpu().numpy()#F.softmax(y_pred).cpu().numpy()


0.895 precision. 0.9624 recall. 0.9275 F1.
Epoch 1/2 	 loss=1.4437 	 val_loss=0.2718  	 val_acc=0.9275  	 time=68.31s
0.92 precision. 0.9634 recall. 0.9412 F1.
Epoch 2/2 	 loss=0.2461 	 val_loss=0.1622  	 val_acc=0.9412  	 time=68.55s
this loop took 0.04 hours to run


In [27]:
model = torch.load('{}_model'.format("cnn"), map_location="cpu")

In [28]:
# make the test tensors for text to embeddings X and for categories/products Y
test_data = pd.read_csv('test_all_processed.csv')
test_description = [literal_eval(s) for s in test_data.description.tolist()]

x_val = []
for i in test_description:
    temp = []
    for j in i:
        # some of the Amazon tokens don't appear in the training data
        # so use try/except to skip any unaccounted for Amazon test data tokens
        try: temp.append(myembeddings[j])
        except: pass
    x_val.append(temp)

In [29]:
new_xv = []
for x in x_val:
    if len(x) > maxlen: new_xv.append(np.array(x[:maxlen]))
    else: new_xv.append(np.array(list(x) + [0]*(maxlen-len(x)) )  )
    
new_xv = np.array(new_xv)
        

In [30]:
x_cv = torch.tensor(new_xv, dtype=torch.long)
y_pred = model(x_cv).detach()
val_preds = F.softmax(y_pred).cpu().numpy()#y_pred.cpu().numpy()#F.softmax(y_pred).cpu().numpy()
# Check Accuracy
pred_ = [np.where(i>0.5) for i in val_preds]#[np.where(i>0) for i in val_preds]

C:\Users\Owner\AppData\Local\Temp\ipykernel_18476\2977559241.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  val_preds = F.softmax(y_pred).cpu().numpy()#y_pred.cpu().numpy()#F.softmax(y_pred).cpu().numpy()


In [31]:
pred_ = [np.where(i>0.5)[0][0] for i in val_preds]#[np.where(i>0) for i in val_preds]
pred_

[1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,


In [32]:
print(sum(pred_))

1166


In [33]:
pd.DataFrame({'id':test_data.id.tolist(), 'target':pred_}).to_csv('submission.csv', index=False)

In [34]:
print('Done.')

Done.


###### 